In [ ]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import os
import random
import numpy as np
import time
import shutil

# SCRAP BY BRAND

In [ ]:
# site to scrap
source = 'bikez'

# url to scrap
url = 'https://bikez.com/brands/index.php'
response = requests.get(url)
    
soup = BeautifulSoup(response.content, "html.parser")

for brand_ in soup.select('#pagecontent > table:nth-child(9)')[0].find_all('tr')[1:][4:5]: #to delete
    try:
        brand = brand_.text.split('motorcycles')[0].strip()
        url = brand_.find('a').get('href')
        brand_url = 'https://bikez.com' + url.replace('..','')
        
        print(brand)
        print(brand_url)
        
        response = requests.get(brand_url)
        soup = BeautifulSoup(response.content, "html.parser")
        
        for model_ in soup.find('table', class_='zebra').find_all('tr', class_="even"):
            model = model_.find_all('td')[1].find('a').text.replace(brand,'').split('(')[0].strip()
            url = model_.find_all('td')[1].find('a').get('href')
            model_url = 'https://bikez.com' + url.replace('..','')
            print('--',model)
            print('--',model_url)
            
            response = requests.get(model_url)
            soup = BeautifulSoup(response.content, "html.parser")
            
            if 'motorcycles' in url:
                
                bike_url = model_url
                print('---',bike_url)

                response_bike = requests.get(bike_url)
                soup_bike = BeautifulSoup(response_bike.content, "html.parser")

                features = ['Brand:',
                            'Model:',
                            'Submodel:',
                            'Year:',
                            'Category:', 
                            'Engine type:',
                            'Displacement:',
                            'Power:',
                            'Torque:',
                            'Compression:',
                            'Bore x stroke',
                            'Fuel system',
                            'Cooling system:',
                            'Shaft drive (cardan)',
                            'Wheels:',
                            'Dry weight:',
                            'Power/weight ratio:']

                features_dict = {
                    'Brand:':[brand],
                    'Model:':[np.nan], 
                    'Submodel:':[np.nan], 
                    'Year:':[np.nan],
                    'Category:':[np.nan], 
                    'Engine type:':[np.nan],
                    'Displacement:':[np.nan],
                    'Power:':[np.nan],
                    'Torque:':[np.nan],
                    'Compression:':[np.nan],
                    'Bore x stroke':[np.nan],
                    'Fuel system':[np.nan],
                    'Cooling system:':[np.nan],
                    'Shaft drive (cardan)':[np.nan],
                    'Wheels:':[np.nan],
                    'Dry weight:':[np.nan],
                    'Power/weight ratio:':[np.nan]}

                rows = soup_bike.find_all('td')
                data = []

                for row in rows:
                    data.append(row.text)

                for feature in features:
                    try:
                        position = data.index(feature)+1
                        features_dict[feature]= [data[position]]
                    except:
                        pass

                df = pd.DataFrame(features_dict)
                csvname = f"bike/{features_dict['Model:'][0]}-{features_dict['Year:'][0]}.csv"
                df.to_csv(csvname, index=False)
            
            elif '/models/' in model_url:
                for submodels_ in soup.find_all('p'):
                    url = submodels_.find('a').get('href')
                    bike_url = 'https://bikez.com' + url.replace('..','')
                    print('---', bike_url)

                    response_bike = requests.get(bike_url)
                    soup_bike = BeautifulSoup(response_bike.content, "html.parser")

                    features = ['Brand:',
                                'Model:',
                                'Submodel:',
                                'Year:',
                                'Category:', 
                                'Engine type:',
                                'Displacement:',
                                'Power:',
                                'Torque:',
                                'Compression:',
                                'Bore x stroke',
                                'Fuel system',
                                'Cooling system:',
                                'Shaft drive (cardan)',
                                'Wheels:',
                                'Dry weight:',
                                'Power/weight ratio:']

                    features_dict = {
                        'Brand:':[brand],
                        'Model:':[np.nan], 
                        'Submodel:':[model], 
                        'Year:':[np.nan],
                        'Category:':[np.nan], 
                        'Engine type:':[np.nan],
                        'Displacement:':[np.nan],
                        'Power:':[np.nan],
                        'Torque:':[np.nan],
                        'Compression:':[np.nan],
                        'Bore x stroke':[np.nan],
                        'Fuel system':[np.nan],
                        'Cooling system:':[np.nan],
                        'Shaft drive (cardan)':[np.nan],
                        'Wheels:':[np.nan],
                        'Dry weight:':[np.nan],
                        'Power/weight ratio:':[np.nan]}

                    rows = soup_bike.find_all('td')
                    data = []

                    for row in rows:
                        data.append(row.text)

                    for feature in features:
                        try:
                            position = data.index(feature)+1
                            features_dict[feature]= [data[position]]
                        except:
                            pass

                    df = pd.DataFrame(features_dict)
                    csvname = f"bike/{features_dict['Model:'][0]}-{features_dict['Year:'][0]}.csv"
                    df.to_csv(csvname, index=False)
                    
            
                      
    except:
        pass


# BUILD DATAFRAME AND CLEAN

In [ ]:
'''history = pd.DataFrame(
    data = {'Brand':[], 'Model:':[],'Submodel:':[], 'Year:':[], 'Category:':[], 'Engine type:':[],
       'Displacement:':[], 'Power:':[], 'Torque:':[], 'Compression:':[], 'Bore x stroke':[],
       'Fuel system':[], 'Cooling system:':[], 'Shaft drive (cardan)':[], 'Wheels:':[],
       'Dry weight:':[], 'Power/weight ratio:':[]})

history.to_csv("bikez_raw.csv", index=False)

for filename in [file for file in os.listdir('bike') if file.endswith(".csv")]:

    df = pd.read_csv(f"bike/{filename}")
    history = history.append(df)
    
    # move file to vault after process
    # source path
    source_folder = f"bike/{filename}"
    # destination path
    destination = f"bike/vault/{filename}"
    # Move the content of
    # source to destination
    shutil.move(source_folder, destination)

history.to_csv("bikez_raw.csv", index=False)'''

In [85]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import os
import random
import numpy as np
import time

df = pd.read_csv("bikez_raw.csv")
df.drop(columns=['Brand'], inplace=True)

df =  df [['Brand:', 'Submodel:', 'Model:',  'Year:', 'Category:', 'Engine type:',
       'Displacement:', 'Power:', 'Torque:', 'Compression:', 'Bore x stroke',
       'Fuel system', 'Cooling system:', 'Shaft drive (cardan)', 'Wheels:',
       'Dry weight:', 'Power/weight ratio:']]

new_columns_name = ['brand', 'model', 'submodel', 'year', 'category', 'engine_type',
                    'displacement', 'power', 'torque', 'compression', 'bore_x_stroke',
                    'fuel_system', 'cooling_system', 'shaft_drive', 'wheels',
                    'dry_weight', 'power_weight_ratio']

df.columns = new_columns_name

df.model.fillna(df.submodel, inplace=True)

df.year = df.year.astype(int)

df['power_weight_ratio'] = df['power_weight_ratio'].str.replace(" HP/kg", "")

def parser(to_clean, delimiter):
    try:
        if type(to_clean)==float:
            return np.nan
        cleaned = to_clean.split(f' {delimiter}')[0].replace(",","")
        return float(cleaned)
    except:
        return np.nan

df.displacement = df.apply(lambda x: parser(x['displacement'],'ccm'), axis=1)
df.power = df.apply(lambda x: parser(x['power'],'HP'), axis=1)
df.torque = df.apply(lambda x: parser(x['torque'],'Nm'), axis=1)
df['dry_weight'] = df.apply(lambda x: parser(x['dry_weight'],'kg'), axis=1)

def clean_model(submodel, brand):
    return str(submodel).replace(str(brand), '').strip()

df['submodel'] = df.apply(lambda x: clean_model(x.submodel, x.brand), axis=1)
df['model'] = df.apply(lambda x: clean_model(x.model, x.brand), axis=1)


df.submodel.fillna('-', inplace=True);

def replace_empty_submodel(model, submodel):
    if submodel == '-':
        return model
    return submodel

#cleaning
to_clean = ['submodel', 'model', 'brand', 'category', 'engine_type', 'cooling_system', 'shaft_drive', 'wheels']

for feature in to_clean:
    print(feature)
    df[feature] = df[feature].str.lower().str.strip()

#puncutation
df.submodel = df.submodel.str.replace(r"\(.*\)","").str.replace('[^\w\s]','')
df.model = df.model.str.replace(r"\(.*\)","").str.replace('[^\w\s]','')
df.brand = df.brand.str.replace(r"\(.*\)","").str.replace('[^\w\s]','')
df.shaft_drive = df.shaft_drive.str.replace(":","")

# engine type
def parser_engine_type(engine_type):
    try:
        return engine_type.split(',')[0]
    except:
        return np.nan

df['engine_type'] = df.apply(lambda x: parser_engine_type(x['engine_type']), axis=1)
#clean engine type
df['engine_type'] = df['engine_type'].str.lower().str.strip()

df.columns = [x+"_db" for x in df.columns]
df.rename(columns={'displacement_db': 'engine_size_db', 'type_db': 'category_db'},  inplace=True)

df.to_csv('bikez.csv', index=False)

submodel
model
brand
category
engine_type
cooling_system
shaft_drive
wheels


<ipython-input-85-e40c4f1ced60>:69: FutureWarning: The default value of regex will change from True to False in a future version.
  df.submodel = df.submodel.str.replace(r"\(.*\)","").str.replace('[^\w\s]','')
<ipython-input-85-e40c4f1ced60>:70: FutureWarning: The default value of regex will change from True to False in a future version.
  df.model = df.model.str.replace(r"\(.*\)","").str.replace('[^\w\s]','')
<ipython-input-85-e40c4f1ced60>:71: FutureWarning: The default value of regex will change from True to False in a future version.
  df.brand = df.brand.str.replace(r"\(.*\)","").str.replace('[^\w\s]','')


In [ ]:
'''

# df.model_db = df.apply(lambda x: replace_empty_model(x.model, x.model_submodel), axis=1)

#def reverse_sentence(sentence):
    return " ".join(sentence.split()[::-1])

#df['model_submodel_inv'] = df['model_submodel'].apply(reverse_sentence)
#df['model_inv'] = df['model'].apply(reverse_sentence)

#df['raw_brand'] = df.brand.str.lower()
#df['raw_model'] = df.model.str.lower()
#df['raw_model_submodel'] = df.model_submodel.str.lower()
#df['raw_brand_model'] = df.apply(lambda x: str(x['brand']).lower() + " " + str(x['model']).lower() ,axis=1)
#df['raw_brand_submodel'] = df.apply(lambda x: str(x['brand']).lower() + " " + str(x['model_submodel']).lower(),axis=1)

# brand and model cleaning
#df.model_submodel_inv = df.model_submodel_inv.str.lower()
df.model_submodel = df.model_submodel.str.lower()
df.model = df.model.str.lower()
df.model_inv = df.model_inv.str.lower()
df.brand = df.brand.str.lower()
df.model_submodel_inv = df.model_submodel_inv.str.strip()
df.model_submodel = df.model_submodel.str.strip()
df.model = df.model.str.strip()
df.model_inv = df.model_inv.str.strip()
df.brand = df.brand.str.strip()
df.model_submodel_inv = df.model_submodel_inv.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
df.model_submodel = df.model_submodel.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
df.model = df.model.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
df.model_inv = df.model_inv.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
df.brand = df.brand.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")

#clean category
df.category = df.category.str.lower().str.strip().str.replace('motorcycle', '')

# engine type
def parser_engine_type(engine_type):
    try:
        return engine_type.split(',')[0]
    except:
        return np.nan

df['engine_type'] = df.apply(lambda x: parser_engine_type(x['engine_type']), axis=1)
#clean engine type
df['engine_type'] = df['engine_type'].str.lower().str.strip()

df.columns = [x+"_db" for x in df.columns]
df.rename(columns={'displacement_db': 'engine_size_db', 'type_db': 'category_db'},  inplace=True)
#'raw_brand_db', 'raw_model_db', 'raw_model_submodel_db','raw_brand_model_db', 'raw_brand_submodel_db',
df = df[['brand_db','model_db', 'model_inv_db', 'model_submodel_db','model_submodel_inv_db', 'year_db', 'category_db', 'engine_type_db',
       'engine_size_db', 'power_db', 'torque_db', 'compression_db',
       'cooling_system_db', 'dry_weight_db', 'power/weight_ratio_db']]

df['model_size_db'] = df.apply(lambda x: str(x['model_db'])+str(x['engine_size_db']).replace('.0',''),axis=1)
df['model_size_inv_db'] = df['model_size_db'].apply(reverse_sentence)

df.model_db.fillna('-', inplace=True)

def empty_model(model, submodel):
    if model == '-':
        return submodel
    return model

df.model_submodel_db = df.apply(lambda x: empty_model(x.model_db, x.model_submodel_db), axis=1)

df.to_csv('bikez.csv')

'''

# IMPORT AND VISUALIZE

In [80]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import os
import random
import numpy as np
import time

In [81]:
df = pd.read_csv('bikez.csv')
df.dropna(subset=['model_db'], inplace=True)
df.to_csv('bikez.csv', index=False)

In [82]:
df.shape

(35216, 17)

In [83]:
df.isnull().sum()

brand_db                     0
model_db                     0
submodel_db                 26
year_db                      0
category_db                  0
engine_type_db              10
engine_size_db             866
power_db                 11367
torque_db                20367
compression_db           14123
bore_x_stroke_db         35216
fuel_system_db           35216
cooling_system_db         4039
shaft_drive_db           31243
wheels_db                26891
dry_weight_db            14235
power_weight_ratio_db    21023
dtype: int64

In [84]:
df

,brand_db,model_db,submodel_db,year_db,category_db,engine_type_db,engine_size_db,power_db,torque_db,compression_db,bore_x_stroke_db,fuel_system_db,cooling_system_db,shaft_drive_db,wheels_db,dry_weight_db,power_weight_ratio_db
0,honda,rebel 250,rebel 250,2002,custom / cruiser,twin,233.8,17.5,17.3,9.2:1,NaN,NaN,NaN,NaN,NaN,141.0,0.1241
1,honda,gl 1800 gold wing,gold wing audio comfort navi xm abs,2012,touring,six cylinder boxer,1832.0,NaN,NaN,9.8:1,NaN,NaN,liquid,clutch,NaN,NaN,NaN
2,aprilia,tx 311,tx 311 m,1986,trial,single cylinder,277.0,17.0,NaN,8.5:1,NaN,NaN,liquid,NaN,NaN,NaN,NaN
3,honda,crf 450,crf450r,2010,cross / motocross,single cylinder,449.0,NaN,NaN,12.0:1,NaN,NaN,liquid,NaN,NaN,NaN,NaN
4,peugeot,vivacity 50,vivacity 3,2012,scooter,single cylinder,49.1,4.8,4.7,NaN,NaN,NaN,air,clutch,NaN,95.0,0.0501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35211,aprilia,dorsoduro 1200,dorsoduro 1200,2015,sport touring,v2,1197.0,130.0,115.0,12.0:1,NaN,NaN,liquid,NaN,NaN,212.0,0.6132
35212,suzuki,gs 500,gs 500 e,1991,naked bike,twin,487.0,46.0,NaN,9.0:1,NaN,NaN,air,NaN,NaN,NaN,NaN
35213,kymco,agility 50,agility 50,2019,scooter,single cylinder,49.0,1.9,3.1,8.0:1,NaN,NaN,air,driveline,NaN,92.5,0.0205
35214,kawasaki,ninja 300,ninja 300 se,2014,sport,twin,296.0,NaN,NaN,10.6:1,NaN,NaN,liquid,NaN,NaN,NaN,NaN
